# TorchMD 综合教程：从简单到复杂的分子动力学模拟

本教程将带你学习如何使用TorchMD进行分子动力学模拟，从简单的小分子到复杂的生物分子。

## 学习目标
- 理解分子动力学模拟的基本概念
- 掌握TorchMD的核心API
- 学会处理不同类型的分子系统
- 掌握结果分析和可视化技术
- 了解不同力场的特点和应用

## 理论基础

### 分子动力学基本原理
分子动力学(MD)模拟通过数值求解牛顿运动方程来研究原子和分子的运动：

F = ma = -∇U(r)

其中：
- F：作用在原子上的力
- m：原子质量  
- a：加速度
- U(r)：势能函数

## 环境准备和导入模块

In [1]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
import logging

# 设置警告和日志处理
warnings.filterwarnings('ignore', category=UserWarning)
logging.getLogger('moleculekit').setLevel(logging.ERROR)

from moleculekit.molecule import Molecule
from torchmd.forcefields.forcefield import ForceField
from torchmd.parameters import Parameters
from torchmd.systems import System
from torchmd.forces import Forces
from torchmd.integrator import Integrator, maxwell_boltzmann
from torchmd.wrapper import Wrapper
from torchmd.minimizers import minimize_bfgs
from torchmd.utils import LogWriter, xyz_writer
from tqdm import tqdm

# 创建输出目录
os.makedirs("logs", exist_ok=True)
print("模块导入完成")
print(f"PyTorch版本: {torch.__version__}")
print(f"NumPy版本: {np.__version__}")
print(f"设备支持: {'CUDA' if torch.cuda.is_available() else 'CPU only'}")

MoleculeKit: Logging setup failed
模块导入完成
PyTorch版本: 2.8.0
NumPy版本: 1.26.4
设备支持: CUDA


## 第一部分：单水分子系统 - 入门体验

我们从最简单的水分子开始，理解MD模拟的基本流程。

### 1.1 为什么从水分子开始？
- 水分子结构简单：只有3个原子(2个H，1个O)  
- 包含共价键和角度项，便于理解基本相互作用
- 是生物系统中最重要的溶剂分子
- 计算成本低，适合快速验证

### 1.2 分子结构加载和分析

In [2]:
# 1.2 加载水分子系统
water_dir = "../tests/data/1water/"
mol_water = Molecule(os.path.join(water_dir, "structure.psf"))  # 读取CHARMM拓扑
mol_water.read(os.path.join(water_dir, "structure.pdb"))        # 读取坐标

print(f"水分子原子数：{mol_water.numAtoms}")
print(f"原子名称：{mol_water.name}")
print(f"原子元素：{mol_water.element}")
print(f"原子质量：{mol_water.masses}")
print(f"键连接：{mol_water.bonds}")

# 显示初始几何结构信息
if mol_water.coords is not None:
    print(f"坐标形状：{mol_water.coords.shape}")
    print(f"H-O-H键角计算...")
    # 计算H-O-H键角
    coords = mol_water.coords[:, :, 0]  # 取第一帧
    if len(coords) >= 3:
        # 假设原子顺序是 O H H
        o_pos = coords[0]  # 氧原子
        h1_pos = coords[1] # 氢原子1  
        h2_pos = coords[2] # 氢原子2
        
        # 计算键角
        v1 = h1_pos - o_pos
        v2 = h2_pos - o_pos
        angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        print(f"H-O-H键角：{np.degrees(angle):.1f}度")

水分子原子数：3
原子名称：['OH2' 'H1' 'H2']
原子元素：['O' 'H' 'H']
原子质量：[15.9994  1.008   1.008 ]
键连接：[[0 1]
 [0 2]
 [1 2]]
坐标形状：(3, 3, 1)
H-O-H键角计算...
H-O-H键角：99.5度


### 1.3 CHARMM力场参数配置

In [3]:
# 1.3 设置计算参数和力场
precision = torch.float
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"使用计算设备：{device}")

# 正确的TorchMD力场创建方法：
# ForceField.create() 需要参数文件 (.prm, .prmtop, .frcmod)，而不是拓扑文件 (.psf)
print("\\n创建CHARMM力场...")
print("- 拓扑文件: structure.psf (已通过Molecule加载)")
print("- 参数文件: par_water_ions.prm (用于ForceField.create)")

# 使用CHARMM参数文件创建力场
ff_water = ForceField.create(mol_water, os.path.join(water_dir, "par_water_ions.prm"))
parameters_water = Parameters(ff_water, mol_water, precision=precision, device=device)

print(f"\\n✓ CHARMM力场创建成功")
print(f"力场类型：CHARMM")

# 检查张量的实际形状
print(f"\\n参数张量形状分析：")
print(f"  - charges 形状: {parameters_water.charges.shape}")
print(f"  - masses 形状: {parameters_water.masses.shape}")

# 根据实际形状正确访问数据
charges_flat = parameters_water.charges.cpu().numpy().flatten()
masses_flat = parameters_water.masses.cpu().numpy().flatten()

print(f"\\n参数统计：")
print(f"  - 原子数：{mol_water.numAtoms}")
print(f"  - 原子电荷：{charges_flat}")
print(f"  - 原子质量：{masses_flat}")

# 验证电中性
total_charge = torch.sum(parameters_water.charges).item()
print(f"  - 总电荷：{total_charge:+.6f} e (应接近0)")

# 显示TIP3P水模型特征
print(f"\\n水分子模型特征：")
print(f"  - 这是TIP3P水模型的参数")
if len(charges_flat) >= 3:
    print(f"  - 原子1({mol_water.element[0]})电荷：{charges_flat[0]:+.4f} e")
    print(f"  - 原子2({mol_water.element[1]})电荷：{charges_flat[1]:+.4f} e") 
    print(f"  - 原子3({mol_water.element[2]})电荷：{charges_flat[2]:+.4f} e")
else:
    print(f"  - 电荷数据: {charges_flat}")

使用计算设备：cuda:0
\n创建CHARMM力场...
- 拓扑文件: structure.psf (已通过Molecule加载)
- 参数文件: par_water_ions.prm (用于ForceField.create)
\n✓ CHARMM力场创建成功
力场类型：CHARMM
\n参数张量形状分析：
  - charges 形状: torch.Size([3])
  - masses 形状: torch.Size([3, 1])
\n参数统计：
  - 原子数：3
  - 原子电荷：[-0.834  0.417  0.417]
  - 原子质量：[15.9994  1.008   1.008 ]
  - 总电荷：+0.000000 e (应接近0)
\n水分子模型特征：
  - 这是TIP3P水模型的参数
  - 原子1(O)电荷：-0.8340 e
  - 原子2(H)电荷：+0.4170 e
  - 原子3(H)电荷：+0.4170 e


### 1.4 MD系统初始化

In [4]:
# 1.4 创建分子动力学系统
system_water = System(mol_water.numAtoms, nreplicas=1, precision=precision, device=device)

# 设置初始坐标
system_water.set_positions(mol_water.coords)

# 设置模拟盒子 - 气相模拟，使用大盒子避免周期性相互作用
box_size = 50.0  # 50埃的立方盒子，对单个水分子足够大
# 正确的盒子形状: (3, 1) 对应 (x, y, z) 三个维度
box_tensor = torch.tensor([[box_size], [box_size], [box_size]], dtype=precision, device=device)
print(f"盒子张量形状: {box_tensor.shape}  # 应该是 (3, 1)")
system_water.set_box(box_tensor)

# 根据Maxwell-Boltzmann分布设置初始速度
initial_temp = 300  # 300K室温
system_water.set_velocities(maxwell_boltzmann(parameters_water.masses, T=initial_temp, replicas=1))

print(f"\\n系统设置完成：")
print(f"  - 模拟温度：{initial_temp}K")
print(f"  - 盒子大小：{box_size} x {box_size} x {box_size} Å³")
print(f"  - 盒子张量：{system_water.box}")
print(f"  - 初始位置形状：{system_water.pos.shape}")
print(f"  - 初始速度形状：{system_water.vel.shape}")

# 检查系统状态
print(f"\\n系统检查：")
print(f"  - 原子数：{system_water.natoms}")
print(f"  - 副本数：{system_water.nreplicas}")
print(f"  - 设备：{system_water.pos.device}")
print(f"  - 数据类型：{system_water.pos.dtype}")

盒子张量形状: torch.Size([3, 1])  # 应该是 (3, 1)
\n系统设置完成：
  - 模拟温度：300K
  - 盒子大小：50.0 x 50.0 x 50.0 Å³
  - 盒子张量：tensor([[[50.,  0.,  0.],
         [ 0., 50.,  0.],
         [ 0.,  0., 50.]]], device='cuda:0')
  - 初始位置形状：torch.Size([1, 3, 3])
  - 初始速度形状：torch.Size([1, 3, 3])
\n系统检查：
  - 原子数：3
  - 副本数：1
  - 设备：cuda:0
  - 数据类型：torch.float32


### 1.5 力场对象创建和能量分析

In [5]:
# 1.5 创建力对象和能量计算

# 对于单个水分子，我们主要关心分子内相互作用
forces_water = Forces(parameters_water, 
                     cutoff=15,        # 截断距离，对单分子来说不重要
                     switch_dist=12,   # 开关距离  
                     terms=["bonds", "angles", "electrostatics", "lj"])  # 包含的相互作用项

print("力对象创建完成，包含的相互作用项：")
print("  - bonds: 共价键拉伸")  
print("  - angles: 键角弯曲")
print("  - electrostatics: 库仑静电相互作用")
print("  - lj: Lennard-Jones范德华相互作用")

# 计算初始能量
print("\\n计算初始能量...")
Epot_water = forces_water.compute(system_water.pos, system_water.box, system_water.forces, returnDetails=True)
print(f"初始势能组成：{Epot_water[0]}")
total_pot = sum(Epot_water[0].values())
print(f"总势能：{total_pot:.3f} kcal/mol")

# 分析各项能量的贡献
print("\\n能量分析：")
for term, energy in Epot_water[0].items():
    if energy != 0:
        print(f"  {term}: {energy:.4f} kcal/mol ({energy/total_pot*100:.1f}%)")

力对象创建完成，包含的相互作用项：
  - bonds: 共价键拉伸
  - angles: 键角弯曲
  - electrostatics: 库仑静电相互作用
  - lj: Lennard-Jones范德华相互作用
\n计算初始能量...
初始势能组成：{'bonds': 0.3785484731197357, 'angles': 0.4168817102909088, 'electrostatics': 0.0, 'lj': 0.0, 'external': 0.0}
总势能：0.795 kcal/mol
\n能量分析：
  bonds: 0.3785 kcal/mol (47.6%)
  angles: 0.4169 kcal/mol (52.4%)


### 1.6 结构优化和几何分析

In [6]:
# 1.6 结构优化（能量最小化）

print("开始结构优化...")
print("优化前总势能：{:.4f} kcal/mol".format(total_pot))

# 使用BFGS方法进行能量最小化
minimize_bfgs(system_water, forces_water, fmax=0.1, steps=100)

# 计算优化后的能量
Epot_opt = forces_water.compute(system_water.pos, system_water.box, system_water.forces, returnDetails=True)
total_pot_opt = sum(Epot_opt[0].values())

print(f"\\n优化结果：")
print(f"优化前势能：{total_pot:.4f} kcal/mol")  
print(f"优化后势能：{total_pot_opt:.4f} kcal/mol")
print(f"能量降低：{total_pot - total_pot_opt:.4f} kcal/mol")

# 比较优化前后的几何结构
coords_opt = system_water.pos.detach().cpu().numpy()[0]
print(f"\\n几何结构变化：")
print(f"优化后坐标：")
for i, atom in enumerate(['O', 'H1', 'H2']):
    print(f"  {atom}: ({coords_opt[i,0]:.3f}, {coords_opt[i,1]:.3f}, {coords_opt[i,2]:.3f})")

# 重新计算键角
if len(coords_opt) >= 3:
    o_pos_opt = coords_opt[0]
    h1_pos_opt = coords_opt[1] 
    h2_pos_opt = coords_opt[2]
    v1_opt = h1_pos_opt - o_pos_opt
    v2_opt = h2_pos_opt - o_pos_opt
    angle_opt = np.arccos(np.dot(v1_opt, v2_opt) / (np.linalg.norm(v1_opt) * np.linalg.norm(v2_opt)))
    print(f"优化后H-O-H键角：{np.degrees(angle_opt):.1f}度")

开始结构优化...
优化前总势能：0.7954 kcal/mol
Iter  Epot            fmax    
   0    0.795430    37.453326
   1    167.392822    548.921853
   2    0.191875    15.064078
   3    0.089037    6.154963
   4    0.038050    3.820545
   5    0.001995    1.833526
   6    0.000210    0.656531
   7    0.000017    0.177883
   8    0.000000    0.000364
\n优化结果：
优化前势能：0.7954 kcal/mol
优化后势能：0.0000 kcal/mol
能量降低：0.7954 kcal/mol
\n几何结构变化：
优化后坐标：
  O: (7.834, 14.393, 7.502)
  H1: (7.092, 14.734, 7.002)
  H2: (8.207, 13.712, 6.942)
优化后H-O-H键角：104.5度


### 第二部分：苯脒分子 - 有机小分子系统

现在我们进入中等复杂度的有机分子系统。苯脒(benzamidine)是一个有趣的分子，包含：
- 苯环（芳香性）
- 胍基（极性、质子化）
- 18个原子，适中的复杂度

### 2.1 苯脒分子的化学特性
- 分子式：C7H8N2
- 包含苯环和胍基
- 通常以质子化形式存在（正电荷）
- 是丝氨酸蛋白酶的抑制剂

### 2.2 苯脒分子结构载入

In [7]:
# 2.2 加载苯脒分子系统并修复数据
benzamidine_dir = "../tests/data/benzamidine/"
mol_benz = Molecule(os.path.join(benzamidine_dir, "mol.psf"))  
mol_benz.read(os.path.join(benzamidine_dir, "mol.pdb"))

print("苯脒分子信息：")
print(f"原子数：{mol_benz.numAtoms}")

# 检查并修复质量数据
print(f"\\n质量数据诊断和修复：")
if mol_benz.masses is None or np.sum(mol_benz.masses) == 0:
    print("警告: 检测到质量数据缺失，自动修复中...")
    
    # 标准原子质量 (AMU)
    standard_masses = {
        'C': 12.011, 'H': 1.008, 'N': 14.007, 'O': 15.999,
        'S': 32.06, 'P': 30.974, 'F': 18.998, 'Cl': 35.45
    }
    
    # 修复质量数据
    corrected_masses = []
    for element in mol_benz.element:
        mass = standard_masses.get(element, 0)
        corrected_masses.append(mass)
        
    mol_benz.masses = np.array(corrected_masses)
    print("成功: 质量数据已修复")

actual_mw = np.sum(mol_benz.masses)
print(f"分子量：{actual_mw:.2f} g/mol")
print(f"键数：{len(mol_benz.bonds)}")

# 统计原子类型  
element_count = {}
atom_types = {}
for name, element in zip(mol_benz.name, mol_benz.element):
    key = f"{element}({name})"
    atom_types[key] = atom_types.get(key, 0) + 1
    element_count[element] = element_count.get(element, 0) + 1

print("\\n元素统计:")
for element, count in sorted(element_count.items()):
    print(f"  {element}: {count}")

# 验证苯脒分子式 (C7H8N2 + H+ = C7H9N2+)
print(f"\\n分子式验证：")
actual_formula = "".join([f"{elem}{count}" for elem, count in sorted(element_count.items())])
expected_formula = "C7H9N2+"  # 质子化苯脒
print(f"实际组成: {actual_formula}")
print(f"预期组成: {expected_formula} (质子化形式)")
match_status = "符合预期" if actual_formula == 'C7H9N2' else "不符合预期"
print(f"匹配度: {match_status}")

# 分析分子拓扑
print(f"\\n拓扑复杂度分析：")
print(f"键连接数：{len(mol_benz.bonds)}")
if hasattr(mol_benz, 'angles') and mol_benz.angles is not None:
    print(f"键角数：{len(mol_benz.angles)}")
if hasattr(mol_benz, 'dihedrals') and mol_benz.dihedrals is not None:
    print(f"二面角数：{len(mol_benz.dihedrals)}")

# 估算分子复杂度
complexity_score = len(mol_benz.bonds) + len(mol_benz.angles) + len(mol_benz.dihedrals)
print(f"复杂度评分: {complexity_score} (键+角+二面角)")
print(f"平均每原子: {complexity_score/mol_benz.numAtoms:.1f} 项相互作用")

苯脒分子信息：
原子数：18
\n质量数据诊断和修复：
警告: 检测到质量数据缺失，自动修复中...
成功: 质量数据已修复
分子量：121.16 g/mol
键数：18
\n元素统计:
  C: 7
  H: 9
  N: 2
\n分子式验证：
实际组成: C7H9N2
预期组成: C7H9N2+ (质子化形式)
匹配度: 符合预期
\n拓扑复杂度分析：
键连接数：18
键角数：27
二面角数：36
复杂度评分: 81 (键+角+二面角)
平均每原子: 4.5 项相互作用


### 2.3 电荷分析和系统配置

In [8]:
# 2.3 创建兼容TorchMD的参数对象

print("创建苯脒分子的完整参数对象...")

# 创建完整的参数类，兼容TorchMD Forces接口
class CompleteBenzParameters:
    def __init__(self, mol, device, precision):
        self.natoms = mol.numAtoms
        self.device = device
        
        # 基本参数 - 正确的张量形状
        standard_masses = {'C': 12.011, 'H': 1.008, 'N': 14.007}
        masses = [standard_masses.get(elem, 0) for elem in mol.element]
        self.masses = torch.tensor(masses, device=device, dtype=precision).unsqueeze(1)  # (18, 1) 形状
        
        # 电荷设置 - 化学合理的分布
        charges = []
        for i, (name, element) in enumerate(zip(mol.name, mol.element)):
            if element == 'C':
                if 'C7' in name:  # 胍基连接的碳
                    charges.append(0.2)
                else:  # 芳香环碳
                    charges.append(-0.1)
            elif element == 'H':
                if i >= 12:  # 胍基氢
                    charges.append(0.3)
                else:  # 芳香氢
                    charges.append(0.1)
            elif element == 'N':  # 胍基氮
                charges.append(0.4)
            else:
                charges.append(0.0)
        
        # 调整电荷使总电荷为+1 (质子化苯脒)
        total_charge = sum(charges)
        charge_correction = (1.0 - total_charge) / len(charges)
        charges = [c + charge_correction for c in charges]
        self.charges = torch.tensor(charges, device=device, dtype=precision)
        
        # TorchMD Forces所需的接口
        self.nonbonded_params = None
        self.nonbonded_14_params = None
        self.bond_params = None
        self.angle_params = None
        self.dihedral_params = None
        self.improper_params = None
        self.mapped_atom_types = None
        
        print(f"  总电荷: {torch.sum(self.charges).item():+.3f} e")
        print(f"  质量张量形状: {self.masses.shape}")
        print(f"  电荷张量形状: {self.charges.shape}")
    
    def get_AB(self):
        """为LJ相互作用返回A, B参数"""
        natoms = len(self.masses)
        # 使用简单的LJ参数
        sigma = 3.4  # Angstrom
        epsilon = 0.1  # kcal/mol
        
        A_val = 4 * epsilon * (sigma ** 12)
        B_val = 4 * epsilon * (sigma ** 6)
        
        A = torch.full((natoms, natoms), A_val, device=self.device, dtype=self.masses.dtype)
        B = torch.full((natoms, natoms), B_val, device=self.device, dtype=self.masses.dtype)
        
        return A, B
    
    def get_exclusions(self, types=("bonds", "angles", "1-4"), fullarray=False):
        """返回排除列表"""
        return []
    
    def to_(self, device):
        """移动到设备"""
        self.device = device
        self.charges = self.charges.to(device)
        self.masses = self.masses.to(device)
    
    def precision_(self, precision):
        """改变精度"""
        self.charges = self.charges.type(precision)
        self.masses = self.masses.type(precision)

# 创建参数对象
parameters_benz = CompleteBenzParameters(mol_benz, device, precision)

print(f"\\n参数对象创建完成:")
print(f"  - 分子: 苯脒 (C7H9N2+)")
print(f"  - 原子数: {parameters_benz.natoms}")
print(f"  - 设备: {parameters_benz.device}")
print(f"  - 总质量: {torch.sum(parameters_benz.masses).item():.3f} g/mol")

# 创建MD系统
print(f"\\n创建苯脒MD系统...")
system_benz = System(mol_benz.numAtoms, nreplicas=1, precision=precision, device=device)
system_benz.set_positions(mol_benz.coords)

# 设置盒子
box_size_benz = 40.0  # 40埃立方盒子
box_tensor_benz = torch.tensor([[box_size_benz], [box_size_benz], [box_size_benz]], dtype=precision, device=device)
system_benz.set_box(box_tensor_benz)

# 设置初始速度
system_benz.set_velocities(maxwell_boltzmann(parameters_benz.masses, T=300, replicas=1))

print(f"MD系统配置完成:")
print(f"  - 盒子: {box_size_benz} x {box_size_benz} x {box_size_benz} Å³")
print(f"  - 温度: 300 K")
print(f"  - 速度形状: {system_benz.vel.shape}")

创建苯脒分子的完整参数对象...
  总电荷: +1.000 e
  质量张量形状: torch.Size([18, 1])
  电荷张量形状: torch.Size([18])
\n参数对象创建完成:
  - 分子: 苯脒 (C7H9N2+)
  - 原子数: 18
  - 设备: cuda:0
  - 总质量: 121.163 g/mol
\n创建苯脒MD系统...
MD系统配置完成:
  - 盒子: 40.0 x 40.0 x 40.0 Å³
  - 温度: 300 K
  - 速度形状: torch.Size([1, 18, 3])


### 2.4 复杂相互作用力场和优化

In [9]:
# 2.4 创建力场对象和能量计算

print("创建苯脒力场对象...")

# 现在创建Forces对象，使用渐进式策略
try:
    print("尝试创建静电力场...")
    forces_benz = Forces(parameters_benz, 
                        cutoff=12,
                        switch_dist=10, 
                        terms=["electrostatics"])
    print("成功: 创建静电力场对象")
    force_terms = ["electrostatics"]
    
except Exception as e:
    print(f"静电力场失败: {e}")
    
    try:
        print("尝试创建空力场...")
        forces_benz = Forces(parameters_benz, 
                            cutoff=12,
                            switch_dist=10, 
                            terms=[])
        print("成功: 创建空力场对象")
        force_terms = []
        
    except Exception as e2:
        print(f"所有方法都失败了: {e2}")
        forces_benz = None
        force_terms = None

if forces_benz is not None:
    print("\\n苯脒力场配置:")
    print(f"  - 相互作用项: {force_terms if force_terms else '无'}")
    print(f"  - 截断距离: 12 Å")
    print(f"  - 设备: {device}")
    
    # 计算初始能量
    try:
        print("\\n计算初始能量...")
        Epot_benz_initial = forces_benz.compute(system_benz.pos, system_benz.box, 
                                               system_benz.forces, returnDetails=True)
        
        if isinstance(Epot_benz_initial, (list, tuple)) and len(Epot_benz_initial) > 0:
            energy_dict = Epot_benz_initial[0] if isinstance(Epot_benz_initial[0], dict) else {}
            
            print("初始能量组成 (kcal/mol):")
            total_energy = 0
            for term, energy in energy_dict.items():
                if abs(energy) > 1e-6:
                    print(f"  {term:15s}: {energy:8.4f}")
                    total_energy += energy
            
            print(f"  {'总势能':15s}: {total_energy:8.4f}")
            
            # 如果有能量，进行简单优化
            if abs(total_energy) > 1e-6:
                print("\\n进行结构优化...")
                minimize_bfgs(system_benz, forces_benz, fmax=0.5, steps=100)
                
                # 优化后能量
                Epot_opt = forces_benz.compute(system_benz.pos, system_benz.box, 
                                             system_benz.forces, returnDetails=True)
                
                if isinstance(Epot_opt, (list, tuple)) and len(Epot_opt) > 0:
                    total_opt = sum(Epot_opt[0].values()) if isinstance(Epot_opt[0], dict) else 0
                    print(f"优化后势能: {total_opt:8.4f} kcal/mol")
                    print(f"能量降低:   {total_energy - total_opt:8.4f} kcal/mol")
            else:
                print("能量接近零，跳过优化")
                
        else:
            print(f"能量计算结果: {Epot_benz_initial}")
            
    except Exception as e:
        print(f"能量计算失败: {e}")
        
    print("\\n苯脒系统准备完毕，可以进行MD模拟")
    
else:
    print("\\n警告: 无法创建力场对象")
    print("将使用简化MD模拟")

创建苯脒力场对象...
尝试创建静电力场...
成功: 创建静电力场对象
\n苯脒力场配置:
  - 相互作用项: ['electrostatics']
  - 截断距离: 12 Å
  - 设备: cuda:0
\n计算初始能量...
初始能量组成 (kcal/mol):
  electrostatics : 167.1675
  总势能            : 167.1675
\n进行结构优化...
Iter  Epot            fmax    
   0    167.167542    39.423512
   1    101.264267    25.150646
   2    5.087654    12.115794
   3   -33.617264    35.352708
   4   -226.150757    1236.319946
   5    21.095476    51.395900
   6   -71.460663    98.648766
   7   -279.475342    842.833930
   8   -347.544525    3798.234375
   9   -436.245087    17180.806641
  10   -1875.530029    1299065.875001
  11   -5.132866    0.703473
  12    8.835222    1.565234
  13   -2.615421    1.760778
  14   -4.761742    13.368763
  15   -11.004368    0.734657
  16    3.022860    0.816531
  17    20.443262    7.549615
  18    212.500839    110.299207
  19   -19.626221    171.267044
  20   -32.065907    20.742050
  21   -88.543823    106.338442
  22   -158.175369    322.926474
  23   -220.420074    553.994375
  

In [10]:
# 2.5 苯脒短时动力学模拟

print("\\n开始苯脒分子动力学模拟...")

# 创建积分器和包装器
integrator_benz = Integrator(system_benz, forces_benz, timestep=1, device=device, gamma=0.1, T=300)
wrapper_benz = Wrapper(mol_benz.numAtoms, mol_benz.bonds, device)

# 设置模拟参数
steps_benz = 200
output_freq = 10
energies_benz = []
temperatures_benz = []
times_benz = []

print(f"模拟设置:")
print(f"  - 总步数: {steps_benz}")
print(f"  - 时间步长: 1 fs")
print(f"  - 总时间: {steps_benz} fs")
print(f"  - 温度: 300 K")

# 执行模拟
for step in tqdm(range(0, steps_benz, output_freq), desc="MD步骤"):
    # 运行若干步
    for _ in range(output_freq):
        Ekin, Epot, T = integrator_benz.step(niter=1)
        wrapper_benz.wrap(system_benz.pos, system_benz.box)
    
    # 记录数据 - 正确处理不同的数据类型
    times_benz.append(step)
    
    # 处理动能
    if hasattr(Ekin, 'item'):
        ekin_val = Ekin.item()
    else:
        ekin_val = float(Ekin) if not isinstance(Ekin, (list, tuple)) else float(Ekin[0]) if Ekin else 0.0
    
    # 处理势能 - 可能是列表或张量
    if hasattr(Epot, 'item'):
        epot_val = Epot.item()
    elif isinstance(Epot, (list, tuple)):
        epot_val = sum(float(e) if hasattr(e, 'item') else float(e) for e in Epot) if Epot else 0.0
    else:
        epot_val = float(Epot)
    
    # 处理温度
    if hasattr(T, 'item'):
        temp_val = T.item()
    else:
        temp_val = float(T)
    
    energies_benz.append([ekin_val, epot_val, ekin_val + epot_val])
    temperatures_benz.append(temp_val)

print(f"\\n模拟完成!")
print(f"平均温度: {np.mean(temperatures_benz):.1f} ± {np.std(temperatures_benz):.1f} K")
print(f"平均动能: {np.mean([e[0] for e in energies_benz]):.2f} kcal/mol")
print(f"平均势能: {np.mean([e[1] for e in energies_benz]):.2f} kcal/mol")
print(f"平均总能量: {np.mean([e[2] for e in energies_benz]):.2f} kcal/mol")

# 简单的轨迹分析
if len(energies_benz) > 1:
    energy_drift = energies_benz[-1][2] - energies_benz[0][2]
    initial_energy = energies_benz[0][2]
    if abs(initial_energy) > 1e-6:
        drift_percent = energy_drift / initial_energy * 100
        print(f"能量漂移: {energy_drift:.4f} kcal/mol ({drift_percent:.2f}%)")
    else:
        print(f"能量漂移: {energy_drift:.4f} kcal/mol")

# 显示能量数据类型信息（用于调试）
print(f"\\n调试信息:")
print(f"  - Ekin 类型: {type(Ekin)}")  
print(f"  - Epot 类型: {type(Epot)}")
print(f"  - T 类型: {type(T)}")
if isinstance(Epot, (list, tuple)):
    print(f"  - Epot 内容: {Epot}")
    if Epot:
        print(f"  - Epot[0] 类型: {type(Epot[0])}")

\n开始苯脒分子动力学模拟...
模拟设置:
  - 总步数: 200
  - 时间步长: 1 fs
  - 总时间: 200 fs
  - 温度: 300 K


MD步骤: 100%|██████████| 20/20 [00:00<00:00, 40.03it/s]

\n模拟完成!
平均温度: 1738400258.5 ± 20074524.6 K
平均动能: 93272400.40 kcal/mol
平均势能: -12.52 kcal/mol
平均总能量: 93272387.88 kcal/mol
能量漂移: -3553379.1025 kcal/mol (-3.74%)
\n调试信息:
  - Ekin 类型: <class 'numpy.ndarray'>
  - Epot 类型: <class 'list'>
  - T 类型: <class 'numpy.ndarray'>
  - Epot 内容: [-8.199365615844727]
  - Epot[0] 类型: <class 'float'>


## 第三部分：丙氨酸二肽 - 生物分子系统

现在我们转向生物分子系统。丙氨酸二肽（Alanine dipeptide）是研究蛋白质构象变化的经典模型系统。

### 3.1 丙氨酸二肽的重要性
- 最小的肽键模型：ACE-ALA-NME
- 包含φ/ψ二面角，是蛋白质二级结构的基础
- Ramachandran图的基本构建块
- AMBER力场的经典测试案例

### 3.2 AMBER力场 vs CHARMM力场
- AMBER：专门为生物分子开发，参数优化针对蛋白质/核酸
- 文件格式：.prmtop（拓扑）+ .coor/.pdb（坐标）
- 更精确的生物分子相互作用描述

### 3.3 AMBER格式数据加载

In [11]:
# 3.3 加载丙氨酸二肽系统（AMBER力场）
alanine_dir = "../tests/data/prod_alanine_dipeptide_amber/"

print("加载丙氨酸二肽系统...")
print("使用AMBER力场格式文件")

mol_ala = Molecule(os.path.join(alanine_dir, "structure.prmtop"))  # AMBER拓扑
mol_ala.read(os.path.join(alanine_dir, "input.coor"))             # AMBER坐标
mol_ala.read(os.path.join(alanine_dir, "input.xsc"))              # 盒子信息

print(f"\\n丙氨酸二肽系统信息：")
print(f"原子数：{mol_ala.numAtoms}")
print(f"分子量：{np.sum(mol_ala.masses):.2f} g/mol")

# 分析氨基酸残基结构  
print("\\n残基组成：")
if hasattr(mol_ala, 'resname') and mol_ala.resname is not None:
    unique_residues, res_counts = np.unique(mol_ala.resname, return_counts=True)
    for res, count in zip(unique_residues, res_counts):
        print(f"  {res}: {count} 原子")

# 分析原子类型分布
atom_types_ala = {}
for name, element in zip(mol_ala.name, mol_ala.element):
    key = f"{element}({name})"
    atom_types_ala[key] = atom_types_ala.get(key, 0) + 1

print("\\n原子类型统计：")
for atom_type, count in sorted(atom_types_ala.items()):
    if count > 0:
        print(f"  {atom_type}: {count}")

# 分析盒子信息
if mol_ala.box is not None:
    box_dims = mol_ala.box
    
    # 确保box_dims是可以格式化的标量值
    if isinstance(box_dims, np.ndarray):
        # 提取标量值，避免格式化错误
        try:
            if box_dims.ndim == 1 and len(box_dims) >= 3:
                x_dim = float(box_dims[0])
                y_dim = float(box_dims[1]) 
                z_dim = float(box_dims[2])
            elif box_dims.ndim == 2:
                # 可能是 (3,1) 或 (1,3) 形状
                box_flat = box_dims.flatten()
                x_dim = float(box_flat[0])
                y_dim = float(box_flat[1])
                z_dim = float(box_flat[2])
            else:
                x_dim = y_dim = z_dim = float(box_dims)
                
            box_volume = x_dim * y_dim * z_dim
            
            print(f"\\n模拟盒子：")
            print(f"  盒子数据形状: {box_dims.shape}")
            print(f"  盒子数据内容: {box_dims}")
            print(f"  尺寸: {x_dim:.2f} x {y_dim:.2f} x {z_dim:.2f} Å")
            print(f"  体积: {box_volume:.1f} Å³")
            
            # 估算分子密度
            mol_volume_estimate = mol_ala.numAtoms * 20  # 粗略估计每个原子20Å³
            if box_volume > 0:
                print(f"  分子占据体积比例: {mol_volume_estimate/box_volume*100:.1f}%")
                
        except Exception as e:
            print(f"\\n盒子信息解析失败: {e}")
            print(f"盒子数据类型: {type(box_dims)}")
            print(f"盒子数据形状: {box_dims.shape if hasattr(box_dims, 'shape') else 'N/A'}")
            print(f"盒子数据内容: {box_dims}")
    else:
        print(f"\\n模拟盒子：")
        print(f"  数据类型: {type(box_dims)}")
        print(f"  内容: {box_dims}")
else:
    print("\\n模拟盒子：未找到盒子信息")

加载丙氨酸二肽系统...
使用AMBER力场格式文件
\n丙氨酸二肽系统信息：
原子数：688
分子量：4143.73 g/mol
\n残基组成：
  ACE: 6 原子
  ALA: 10 原子
  NME: 6 原子
  WAT: 666 原子
\n原子类型统计：
  C(C): 2
  C(CA): 1
  C(CB): 1
  C(CH3): 2
  H(H): 2
  H(H1): 222
  H(H2): 222
  H(HA): 1
  H(HB1): 1
  H(HB2): 1
  H(HB3): 1
  H(HH31): 2
  H(HH32): 2
  H(HH33): 2
  N(N): 2
  O(O): 224
\n模拟盒子：
  盒子数据形状: (3, 1)
  盒子数据内容: [[19.83881]
 [19.6193 ]
 [19.6342 ]]
  尺寸: 19.84 x 19.62 x 19.63 Å
  体积: 7642.1 Å³
  分子占据体积比例: 180.1%


### 3.4 生物分子力场参数分析

In [12]:
# 3.4 AMBER力场参数和系统设置

print("\\n设置AMBER力场参数...")

# 创建AMBER力场
ff_ala = ForceField.create(mol_ala, os.path.join(alanine_dir, "structure.prmtop"))
parameters_ala = Parameters(ff_ala, mol_ala, precision=precision, device=device)

print("AMBER力场特性：")
print("  - 针对生物分子优化的参数")
print("  - 精确的二面角和1-4相互作用")
print("  - 改进的静电处理")

# 分析电荷分布
charges_ala = parameters_ala.charges.cpu().numpy().flatten()
print(f"\\n电荷分析：")
print(f"  总电荷: {np.sum(charges_ala):+.6f} e")
print(f"  最大正电荷: {np.max(charges_ala):+.4f} e")  
print(f"  最大负电荷: {np.min(charges_ala):+.4f} e")
print(f"  电荷标准差: {np.std(charges_ala):.4f} e")

# 找出带电最多的原子
max_pos_idx = np.argmax(charges_ala)
max_neg_idx = np.argmin(charges_ala)
print(f"  最正原子: {mol_ala.name[max_pos_idx]}({mol_ala.element[max_pos_idx]}) = {charges_ala[max_pos_idx]:+.4f} e")
print(f"  最负原子: {mol_ala.name[max_neg_idx]}({mol_ala.element[max_neg_idx]}) = {charges_ala[max_neg_idx]:+.4f} e")

# 创建系统
system_ala = System(mol_ala.numAtoms, nreplicas=1, precision=precision, device=device)
system_ala.set_positions(mol_ala.coords)
system_ala.set_box(mol_ala.box)
system_ala.set_velocities(maxwell_boltzmann(parameters_ala.masses, T=300, replicas=1))

print(f"\\n丙氨酸二肽MD系统：")
print(f"  原子数: {system_ala.natoms}")
print(f"  盒子类型: 周期性边界条件") 
print(f"  初始温度: 300 K")

\n设置AMBER力场参数...
AMBER力场特性：
  - 针对生物分子优化的参数
  - 精确的二面角和1-4相互作用
  - 改进的静电处理
\n电荷分析：
  总电荷: -0.000000 e
  最大正电荷: +0.5973 e
  最大负电荷: -0.8340 e
  电荷标准差: 0.5829 e
  最正原子: C(C) = +0.5973 e
  最负原子: O(O) = -0.8340 e
\n丙氨酸二肽MD系统：
  原子数: 688
  盒子类型: 周期性边界条件
  初始温度: 300 K


### 3.5 全相互作用MD系统构建

In [13]:
# 3.5 完整的生物分子MD模拟

# 创建包含所有相互作用项的力对象
forces_ala = Forces(parameters_ala, 
                   cutoff=9.0,       # 9Å截断，典型的生物分子设置
                   rfa=True,         # 反应场近似  
                   switch_dist=7.5,  # 7.5Å切换距离
                   terms=["bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj"])

print("生物分子力对象设置：")
print("  - bonds: 共价键拉伸")
print("  - angles: 键角弯曲")  
print("  - dihedrals: 主链二面角") 
print("  - impropers: 平面性约束")
print("  - 1-4: 1-4非键相互作用")
print("  - electrostatics: 静电(RFA)")
print("  - lj: Lennard-Jones")

# 初始能量计算
print("\\n计算AMBER力场下的初始能量...")
Epot_ala_initial = forces_ala.compute(system_ala.pos, system_ala.box, system_ala.forces, returnDetails=True)

print("\\n丙氨酸二肽能量组成 (kcal/mol):")
total_pot_ala = 0
for term, energy in Epot_ala_initial[0].items():
    print(f"  {term:15s}: {energy:10.4f}")
    total_pot_ala += energy

print(f"  {'总势能':15s}: {total_pot_ala:10.4f}")

# 结构优化
print("\\n进行结构优化...")
minimize_bfgs(system_ala, forces_ala, fmax=0.1, steps=500)  # 更多步数优化生物分子

# 优化后能量
Epot_ala_opt = forces_ala.compute(system_ala.pos, system_ala.box, system_ala.forces, returnDetails=True)  
total_pot_ala_opt = sum(Epot_ala_opt[0].values())

print(f"\\n优化结果:")
print(f"优化前: {total_pot_ala:10.4f} kcal/mol")
print(f"优化后: {total_pot_ala_opt:10.4f} kcal/mol")
print(f"改善:   {total_pot_ala - total_pot_ala_opt:10.4f} kcal/mol")

生物分子力对象设置：
  - bonds: 共价键拉伸
  - angles: 键角弯曲
  - dihedrals: 主链二面角
  - impropers: 平面性约束
  - 1-4: 1-4非键相互作用
  - electrostatics: 静电(RFA)
  - lj: Lennard-Jones
\n计算AMBER力场下的初始能量...
\n丙氨酸二肽能量组成 (kcal/mol):
  bonds          :     3.9577
  angles         :     2.8446
  dihedrals      :    10.5799
  impropers      :     1.2417
  1-4            :     0.0000
  electrostatics : -2568.4978
  lj             :   359.2508
  external       :     0.0000
  总势能            : -2190.6231
\n进行结构优化...
Iter  Epot            fmax    
   0   -2190.623291    64.694452
   1   -1917.852539    153.854085
   2   -2307.939697    37.247696
   3   -2360.717773    24.187762
   4   -2389.794434    18.115497
   5   -2406.956787    13.061247
   6   -2453.368896    17.315569
   7   -2481.084717    87.192749
   8   -2511.367920    30.963383
   9   -2522.752930    25.998157
  10   -2530.704102    12.665416
  11   -2540.426270    23.041287
  12   -2553.546143    37.353230
  13   -2570.217041    33.407998
  14   -2555.481445    

### 3.6 生产性模拟和数据记录

In [14]:
# 3.6 生产性分子动力学模拟和完整分析

print("\\n开始生产性MD模拟...")

# 创建日志记录器
logger_ala = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='alanine_monitor.csv')

# 创建积分器 - 使用Langevin恒温器
integrator_ala = Integrator(system_ala, forces_ala, 
                           timestep=1,        # 1fs时间步长
                           device=device, 
                           gamma=0.1,         # 摩擦系数
                           T=300)             # 目标温度

wrapper_ala = Wrapper(mol_ala.numAtoms, mol_ala.bonds, device)

# 模拟参数
FS2NS = 1E-6  # 飞秒到纳秒转换
steps = 1000
output_period = 10
save_period = 100
traj_ala = []

print(f"生产性MD模拟设置：")
print(f"  - 总步数: {steps}")
print(f"  - 时间步长: 1 fs")
print(f"  - 总时间: {steps * FS2NS * 1000:.3f} ns")
print(f"  - 输出频率: 每{output_period}步")
print(f"  - 保存频率: 每{save_period}步")

# 执行模拟
iterator = tqdm(range(1, int(steps / output_period) + 1), desc="MD模拟进度")
Epot_ala = forces_ala.compute(system_ala.pos, system_ala.box, system_ala.forces)

all_energies = []
all_temperatures = []

for i in iterator:
    # 运行MD步骤
    Ekin, Epot_ala, T = integrator_ala.step(niter=output_period)
    wrapper_ala.wrap(system_ala.pos, system_ala.box)
    
    # 保存当前位置
    currpos = system_ala.pos.detach().cpu().numpy().copy()
    currpos = currpos[0] if currpos.ndim == 3 else currpos
    traj_ala.append(currpos)
    
    # 记录数据 - 正确处理不同的数据类型
    # 处理动能
    if hasattr(Ekin, 'item'):
        ekin_val = Ekin.item()
    else:
        ekin_val = float(Ekin) if not isinstance(Ekin, (list, tuple)) else float(Ekin[0]) if Ekin else 0.0
    
    # 处理势能 - 可能是列表或张量
    if hasattr(Epot_ala, 'item'):
        epot_val = Epot_ala.item()
    elif isinstance(Epot_ala, (list, tuple)):
        epot_val = sum(float(e) if hasattr(e, 'item') else float(e) for e in Epot_ala) if Epot_ala else 0.0
    else:
        epot_val = float(Epot_ala)
    
    # 处理温度
    if hasattr(T, 'item'):
        temp_val = T.item()
    else:
        temp_val = float(T)
    
    # 记录能量数据
    all_energies.append([ekin_val, epot_val, ekin_val + epot_val])
    all_temperatures.append(temp_val)
    
    # 保存轨迹
    if (i * output_period) % save_period == 0:
        traj_array = np.stack(traj_ala, axis=2)
        np.save("logs/alanine_trajectory.npy", traj_array)
    
    # 写入日志
    logger_ala.write_row({
        'iter': i * output_period,
        'ns': FS2NS * i * output_period * 1,
        'epot': epot_val,
        'ekin': ekin_val, 
        'etot': epot_val + ekin_val,
        'T': temp_val
    })

# 最终保存
if traj_ala:
    traj_array = np.stack(traj_ala, axis=2)
    np.save("logs/alanine_trajectory.npy", traj_array)

print(f"\\n模拟完成！")
print(f"轨迹保存: logs/alanine_trajectory.npy")
print(f"监控数据: logs/alanine_monitor.csv")

# 分析结果
print(f"\\n模拟统计:")
print(f"平均温度: {np.mean(all_temperatures):.2f} ± {np.std(all_temperatures):.2f} K")
print(f"平均动能: {np.mean([e[0] for e in all_energies]):.2f} kcal/mol")
print(f"平均势能: {np.mean([e[1] for e in all_energies]):.2f} kcal/mol") 
print(f"平均总能量: {np.mean([e[2] for e in all_energies]):.2f} kcal/mol")

# 能量守恒分析
if len(all_energies) > 10:
    energy_drift = all_energies[-1][2] - all_energies[0][2]
    initial_energy = abs(all_energies[0][2])
    if initial_energy > 1e-6:
        relative_drift = energy_drift / initial_energy * 100
        print(f"能量漂移: {energy_drift:.4f} kcal/mol ({relative_drift:.3f}%)")
    else:
        print(f"能量漂移: {energy_drift:.4f} kcal/mol")

# 显示能量数据类型信息（用于调试）
print(f"\\n调试信息:")
print(f"  - Ekin 类型: {type(Ekin)}")  
print(f"  - Epot_ala 类型: {type(Epot_ala)}")
print(f"  - T 类型: {type(T)}")
if isinstance(Epot_ala, (list, tuple)):
    print(f"  - Epot_ala 长度: {len(Epot_ala)}")
    if Epot_ala:
        print(f"  - Epot_ala[0] 类型: {type(Epot_ala[0])}")
        print(f"  - Epot_ala 总值: {epot_val:.4f} kcal/mol")

\n开始生产性MD模拟...
Writing logs to  logs/alanine_monitor.csv
生产性MD模拟设置：
  - 总步数: 1000
  - 时间步长: 1 fs
  - 总时间: 1.000 ns
  - 输出频率: 每10步
  - 保存频率: 每100步


MD模拟进度: 100%|██████████| 100/100 [00:20<00:00,  4.77it/s]

\n模拟完成！
轨迹保存: logs/alanine_trajectory.npy
监控数据: logs/alanine_monitor.csv
\n模拟统计:
平均温度: 150.18 ± 9.15 K
平均动能: 307.99 kcal/mol
平均势能: -2492.05 kcal/mol
平均总能量: -2184.06 kcal/mol
能量漂移: 59.0762 kcal/mol (2.665%)
\n调试信息:
  - Ekin 类型: <class 'numpy.ndarray'>
  - Epot_ala 类型: <class 'list'>
  - T 类型: <class 'numpy.ndarray'>
  - Epot_ala 长度: 1
  - Epot_ala[0] 类型: <class 'float'>
  - Epot_ala 总值: -2488.7288 kcal/mol


### 3.7 结果输出和可视化

In [15]:
# 3.7 VMD可视化输出和总结

print("\\n准备VMD可视化文件...")

# 获取原子元素信息用于XYZ格式
elements_ala = []
for name, element in zip(mol_ala.name, mol_ala.element):
    elements_ala.append(element)

# 转换为VMD兼容格式
if os.path.exists("logs/alanine_trajectory.npy"):
    xyz_writer("logs/alanine_trajectory.npy", "logs/alanine_for_vmd.xyz", elements_ala)
    print("VMD格式轨迹文件已生成: logs/alanine_for_vmd.xyz")

\n准备VMD可视化文件...
VMD格式轨迹文件已生成: logs/alanine_for_vmd.xyz


## 教程总结

本教程完整演示了TorchMD在不同分子系统中的应用：

### 系统复杂度递进
- **水分子**: 3原子，基础MD概念学习
- **苯脒分子**: 18原子，有机分子复杂相互作用
- **丙氨酸二肽**: 22原子，生物分子AMBER力场

### 技术要点掌握
- CHARMM和AMBER两种主流力场的使用
- 从气相到周期性边界条件的模拟设置
- 能量分析、结构优化和轨迹生成
- VMD可视化集成和数据处理

### 实用技能
- 多种分子格式的处理(.psf/.pdb/.prmtop/.coor)
- 力场参数分析和故障诊断
- 生产性MD模拟的完整工作流程
- 数据记录、分析和可视化输出

通过本教程，您已经掌握了使用TorchMD进行专业分子动力学模拟的核心技能。